In [323]:
using DynamicPolynomials, SparseArrays

println("***Problem setting***")

n=25

println("Number of variables: n=",n)
println("====================")

@polyvar x[1:n]# variables

# random quadratic objective function f


A=sprand(n, n, 0.5)
for i in 1:n
    for j=i:n
        if i == j
            A[j,j]=1
        else
            A[i,j]=sign(A[j,i])
            A[j,i]=A[i,j]
        end
    end
end
A=Matrix(A)


#include("./mat_stability_size$(n).jl")


f=sum(A[i,j]*x[i]*x[j] for i=1:n for j=1:n)

            
g=Vector{Polynomial{true,Float64}}([])
#type of coefficients of each polynomial must be float

m=length(g)
println("Number of inequality constraints: m=",m)
println("====================")            

# unit sphere constraint
h=[1.0-sum(x)] #type of coefficients of each polynomial must be float

l=length(h)

println("Number of equality constraints: l=",l)
println("====================")

***Problem setting***
Number of variables: n=25
Number of inequality constraints: m=0
Number of equality constraints: l=1


In [324]:
include("../src/InterruptedRelax.jl")
using .InterruptedRelax

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterruptedRelax.get_info(x,f,g,h,sparse=false);


k=0

println("Relaxation order: k=",k)

s=26

println("Block size: s=",s)

@time opt_val1,opt_sol=InterruptedRelax.RelaxDense(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,
    lmon_f,supp_f,coe_f,dg,dh,k,s,solver="Mosek",comp_opt_sol=false)

Relaxation order: k=0
Block size: s=26
Maximal matrix size:26
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 351             
  Cones                  : 0               
  Scalar variables       : 352             
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : max 

(0.16396855391107124, Float64[])

In [325]:
1/opt_val1

6.09873037327848

In [326]:
include("../src/InterruptedRelax.jl")
using .InterruptedRelax

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterruptedRelax.get_info(x,f,g,h,sparse=false);


k=1

println("Relaxation order: k=",k)

s=26

println("Block size: s=",s)

@time opt_val1,opt_sol=InterruptedRelax.RelaxDense(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,
    lmon_f,supp_f,coe_f,dg,dh,k,s,solver="Mosek")

Relaxation order: k=1
Block size: s=26
Maximal matrix size:26
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 3276            
  Cones                  : 0               
  Scalar variables       : 2952            
  Matrix variables       : 26              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
GP based matrix reordering started.
GP based matrix reordering terminated.
Problem


(0.16666592623652846, Float64[])

In [327]:
1/opt_val1

6.000026655603396

In [328]:
using SparseArrays

include("../src/InterruptedRelax.jl")
using .InterruptedRelax

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterruptedRelax.get_info(x,f,g,h,sparse=true);

#InterruptedRelax.save_info_sparsePOP(randx,u,n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh)

#include("./sparsePOP3.jl")

using TSSOS

k=1

println("Maximal matrix size: ",binomial(n+k,k))

vars,pop=InterruptedRelax.get_POP(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f)

@time opt,sol,data=TSSOS.cs_tssos_first(pop,vars,k,numeq=l,CS=false,TS=false,solver="Mosek");

Maximal matrix size: 26
************************TSSOS************************
TSSOS is launching...
------------------------------------------------------
The clique sizes of varibles:
[25]
[1]
------------------------------------------------------
Assembling the SDP...
There are 351 affine constraints.
SDP assembling time: 0.000826238 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 351             
  Cones                  : 0               
  Scalar variables       : 27              
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time      

In [329]:
1/opt

-0.6337381446574224

In [330]:
using SparseArrays

include("../src/InterruptedRelax.jl")
using .InterruptedRelax

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterruptedRelax.get_info(x,f,g,h,sparse=true);

#InterruptedRelax.save_info_sparsePOP(randx,u,n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh)

#include("./sparsePOP3.jl")

using TSSOS

k=2

println("Maximal matrix size: ",binomial(n+k,k))

vars,pop=InterruptedRelax.get_POP(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f)

@time opt,sol,data=TSSOS.cs_tssos_first(pop,vars,k,numeq=l,CS=false,TS=false,solver="Mosek");

Maximal matrix size: 351
************************TSSOS************************
TSSOS is launching...
------------------------------------------------------
The clique sizes of varibles:
[25]
[1]
------------------------------------------------------
Assembling the SDP...
There are 23751 affine constraints.
SDP assembling time: 0.12946266 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 23751           
  Cones                  : 0               
  Scalar variables       : 352             
  Matrix variables       : 26              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time    

In [333]:
1/opt

6.058011632233154

In [334]:
include("../src/InterruptedRelax.jl")
using .InterruptedRelax

InterruptedRelax.save_info_mat_stability(n,A)